In [36]:
import openai 
from utils import call_chatgpt_api_all_chats, stream_process_chatgpt_response
from secret import gao_key as key 
from resources.generate_response import analyze_situation_rag, analyze_situation_rag_guidance
import concurrent.futures
import re
import time 
import json 
from benefits.eligibility_check import eligibility_check

openai.api_key = key

# High-level, brief response with vanilla LLM

In [17]:
def get_support_recommendations(user_situation):
    system_prompt = (
        "You are a helpful assistant trained to recommend supportive services or interventions "
        "for individuals based on their personal situation. Consider factors like healthcare access, "
        "housing, mental health, age, isolation, legal status, and physical needs. Be specific and actionable."
    )

    example_input = (
        "A 72-year-old retired man in Morristown, NJ, living in permanent housing and dually enrolled (Medicaid/Medicare), "
        "seeks support for maintaining his physical wellness through better diet and exercise. "
        "He is a non-drug user with ID and no incarceration history but struggles with isolation, "
        "spending most of his days alone in his room, and wants to build social connections."
    )

    example_output = (
        "- Refer to local senior wellness programs through NJ Medicaid plans\n"
        "- Enroll in a community-based exercise group like SilverSneakers\n"
        "- Connect with peer support or friendly visitor programs to reduce isolation\n"
        "- Provide dietary planning support from Medicare-covered nutritionists\n"
        "- Suggest transportation services for seniors to attend community events\n"
        "- Encourage enrollment in group classes at a local senior center"
    )

    user_prompt = f"""\
Given the following individual's situation, list specific and actionable supports they may benefit from.

Example input:
{example_input}

Example output:
{example_output}

Now, given the situation:
{user_situation}

What support can be provided?
"""
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    return response.choices[0].message.content
   


In [18]:
situation = "A 28-year-old woman in Paterson, NJ, is living in temporary housing while working a part-time job and has a physical disability that doesn't prevent her from working entirely but makes it difficult. She has documented immigration status but struggles to balance employment demands with physical wellness and social engagement. She is uncertain enough about her sustained ability to earn enough from part-time work to make it that she is asking about disability benefits and housing subsidies."

recommendations = get_support_recommendations(situation)
print(recommendations)

- Refer to New Jersey state Disability Services to explore eligibility and application assistance for disability benefits (e.g., SSDI or SSI)
- Connect with a social worker or benefits counselor specializing in immigrant populations to navigate disability and housing subsidies
- Provide information on New Jersey's Temporary Assistance for Needy Families (TANF) and Section 8 housing subsidy programs
- Refer to vocational rehabilitation services in NJ that support individuals with disabilities in maintaining or finding employment with accommodations
- Connect with local disability advocacy organizations in Paterson for peer support and resources
- Suggest enrollment in community health or wellness programs adapted for people with physical disabilities
- Link to part-time employment resources offering flexible schedules or remote work options suitable for her physical needs
- Provide info on transportation assistance programs for individuals with disabilities to ease commuting challenges


# Note
- The response above represents the general or baseline output from the vanilla LLM, as discussed earlier. On the front end, it would be ideal to include a “Show More Details” button (or a similar format) for each output. Peer providers could click this button to view additional information related to that specific support item—such as goals, resources, follow-up questions, etc. The output format can be adjusted in the future; this is just a proof of concept for now.
  
- I'm using gpt4.1-mini as the model (newest reasoning model from OpenAI, cheap, 0.40 usd/1M tokens (4o-mini charges 0.15 usd/1M tokens))

  
- Further customization can be implemented. For example, the same button could allow peer providers to choose what type of information they want to view—SMART goals, resources, follow-up questions, or all of the above.

# Meta-reasoning for sophisticated context

In [19]:
# Define meta-reasoning prompt 
def generate_meta_reasoning_prompt(situation: str) -> str:
    return f"""
You are an expert mental health support agent tasked with analyzing a user's provided situation to guide subsequent analysis modules. Your goal is NOT to produce concrete SMART goals, resources, or specific questions directly, but rather to reason conceptually and give clear directions on how each specialized module should approach the situation.

Given the user's situation below, clearly articulate conceptual guidance and direction ONLY for each of the following modules:

1. **SMART Goals Module**: 
   - What key themes or areas should SMART goals focus on, given this situation?
   - Briefly outline what characteristics or considerations the SMART goals should prioritize, without listing actual goals.

2. **Resources Module**:
   - Identify types or categories of resources that are conceptually relevant or could potentially benefit the user.
   - Specify the criteria or conditions to use when searching for relevant resources.

3. **Follow-up Questions Module**:
   - Conceptually suggest what kinds of insightful follow-up questions would be helpful.
   - Highlight themes or areas that need clarification or further exploration.

4. **Benefits Eligibility Module**:
   - Provide high-level guidance on what types of benefits or eligibility considerations should be examined.
   - Mention what contextual clues or criteria would be helpful to determine eligibility.

### User Situation:
{situation}

### Provide structured, conceptual reasoning clearly labeled for each module:
"""

In [20]:
# o3-mini for enhanced reasoning capability
# lower price than 4o but more expensive than 4o-mini ($15 / 1M tokens)
# o3-mini is only used during the reasoning step

def call_llm_reason(messages, model="o3-mini"):
    response = openai.chat.completions.create(
        model=model,
        messages=messages
    )
    return response.choices[0].message.content

def get_meta_reasoning(situation: str):
    meta_prompt = generate_meta_reasoning_prompt(situation)

    messages = [
        {"role": "system", "content": "You are an expert mental health reasoning assistant providing structured conceptual guidance for mental health support modules."},
        {"role": "user", "content": meta_prompt}
    ]

    reasoning_response = call_llm_reason(messages)
    structured_reasoning = parse_reasoning(reasoning_response)
    
    return structured_reasoning

In [21]:
# Parsing function to neatly structure the response
def parse_reasoning(response_text: str):
    modules = ["SMART Goals Module", "Resources Module", "Follow-up Questions Module", "Benefits Eligibility Module"]
    reasoning = {}

    current_module = None
    for line in response_text.splitlines():
        line_stripped = line.strip()
        if any(module in line_stripped for module in modules):
            current_module = next(module for module in modules if module in line_stripped)
            reasoning[current_module] = ""
        elif current_module and line_stripped != "":
            reasoning[current_module] += line_stripped + " "

    # Clean extra spaces
    for key in reasoning:
        reasoning[key] = reasoning[key].strip()

    return reasoning

In [23]:
meta_reasoning_result = get_meta_reasoning(situation)
print("\n=== Structured Meta-Reasoning Output ===\n")
for module, guidance in meta_reasoning_result.items():
    print(f"**{module}**:\n{guidance}\n")


=== Structured Meta-Reasoning Output ===

**SMART Goals Module**:
• Key Themes/Areas: Focus on stability and improvement across work-life balance, physical health management, and social engagement. In addition, consider financial security, particularly in relation to potential disability benefits and housing support. • Characteristics and Considerations: SMART goals should be tailored to measurable outcomes that consider her physical limitations, part-time employment framework, and the necessity for consistent well-being. Goals should be realistic, time-bound, and flexible enough to adjust to her ongoing physical and situational challenges. They must integrate metrics that monitor progress in health, employment, and social recovery while being achievable within her current support system.

**Resources Module**:
• Types/Categories of Resources: Identify resources focused on physical rehabilitation or adaptive services for disabilities, employment support services, local community servi

In [24]:
smart_goal_reasoning = meta_reasoning_result['SMART Goals Module']
resource_reasoning = meta_reasoning_result['Resources Module']
question_reasoning = meta_reasoning_result['Follow-up Questions Module']
benefit_reasonsing = meta_reasoning_result['Benefits Eligibility Module']

In [40]:
mental_health_prompt = open("mental_health/prompts/mental_health_prompt.txt").read()
question_prompt = open("mental_health/prompts/question_prompts.txt").read()
benefit_prompt = open("benefits/prompts/uncertain_prompt_wellness.txt").read()
resource_prompt = open("mental_health/prompts/resource_prompt.txt").read()

# Note
- The “goal” here refers to the specific item the user clicked the “More Information” button on. We are addressing this particular aspect of the vanilla response that the user wants more details about.

- smart_meta_reasoning and followup_meta_reasoning are used as contextual inputs.

- RAG will continue to be used for retrieving relevant resources.


In [41]:
def generate_goal_specific_details(goal, user_situation, smart_meta_reasoning, followup_meta_reasoning, system_prompt, question_prompt, mental_health_prompt):
    messages = [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": f"""\
You are given the following:

1. **User Situation**: 
{user_situation}

2. **Goal of Interest**:
{goal}

3. **SMART Goals Module Meta Reasoning**:
{smart_meta_reasoning}

4. **Follow-up Questions Module Meta Reasoning**:
{followup_meta_reasoning}

Based on the above, do the following:
- {mental_health_prompt}

- {question_prompt}

Please output using the following format:

[SMART Goals]
1. ...
2. ...
...
[/SMART Goals]

[Follow-up Questions]
1. ...
2. ...
...
[/Follow-up Questions]
"""
        }
    ]
    
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=messages
    )
    return response.choices[0].message.content


In [42]:
goal = "Suggest enrollment in community health or wellness programs adapted for people with physical disabilitiess"

system_prompt = (
    "You are a helpful assistant trained to guide peer specialists in supporting individuals' wellness through structured planning. "
    "You provide SMART goals and thoughtful follow-up questions based on the user’s situation and a specific goal of interest. "
    "You do not invent information; you reason clearly and draw only from the provided context. Make goals measurable, achievable, and personalized."
)

# Call the function
output = generate_goal_specific_details(goal, situation, smart_goal_reasoning, question_reasoning, system_prompt, question_prompt, mental_health_prompt)
print(output)


[SMART Goals]  
1. Research and enroll in at least one community health or wellness program adapted for individuals with physical disabilities within the next 30 days in Paterson, NJ. Measure progress by obtaining enrollment confirmation and attending the first session.  
2. Attend at least two sessions per month of the chosen wellness program over the next three months to improve physical health and social engagement. Track attendance and note any improvements in energy levels or social interactions.  
3. Identify and meet with a benefits counselor or social worker within the next 45 days to discuss eligibility and application steps for disability benefits and housing subsidies. Document the meeting outcomes and next steps.  
4. Develop a weekly self-care routine that includes physical wellness activities (e.g., adapted exercises or physical therapy) and social interaction opportunities, dedicating at least 3 hours per week over the next two months. Keep a weekly log to monitor consis

In [43]:
rag_input = f"""
Goal: 
{goal}

User Situation:
{situation}

Guidance for Relevant Resources:
{resource_reasoning}
"""
retrieved_resources = analyze_situation_rag(rag_input, k=15)

/opt/miniconda3/lib/python3.12/site-packages/transformers/models/mpnet/modeling_mpnet.py:1051: UserWarning: cumsum_out_mps supported by MPS on MacOS 13+, please upgrade (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/UnaryOps.mm:425.)
  incremental_indices = torch.cumsum(mask, dim=1).type_as(mask) * mask


In [44]:
retrieved_resources

"The Marketplace (CUMAC), URL: https://www.cumac.org/, Phone: 9737425518, Description: The Marketplace (CUMAC) is a holistic resource designed to combat hunger and its underlying causes by offering groceries and essential items to individuals and families in need. Operating Monday to Friday from 9:30 AM to 3:00 PM, it employs a trauma-informed, appointment-based approach, empowering guests to select their own food while fostering community connections. CUMAC emphasizes diversity and community representation, employing local residents and tailoring programs to address the effects of childhood trauma. The initiative aims not only for immediate relief but also for long-term positive changes in knowledge and behavior.\nCUMAC, URL: https://www.cumac.org/, Phone: 9737425518, Description: CUMAC is dedicated to combating hunger and its underlying causes through a holistic, trauma-informed approach. They provide groceries and basic necessities via an appointment-based Choice Marketplace, empowe

In [59]:
def select_relevant_resources(goal, situation, meta_reasoning_guidance, retrieved_resources):
    prompt = f"""
You are an expert resource recommendation assistant. Given goal that we are addressing, the user's specific situation, conceptual guidance, and a list of potential resources retrieved from a knowledge base, your task is to carefully select and recommend ONLY the most relevant and helpful resources. 

Consider the original situation and the conceptual guidance carefully to ensure selected resources specifically match the user's needs and context.

Do not make anything up. Only select resources from the list that is given to you.

## Goal to be addressed:
{goal}

## Original User Situation:
{situation}

## Conceptual Guidance for Selecting Resources:
{meta_reasoning_guidance}

## Retrieved Resources:
{retrieved_resources}

## Your task:
- Carefully evaluate the retrieved resources in relation to the user's situation and the provided conceptual guidance.
- Select and recommend ONLY the resources that BEST match the user's situation.
- Clearly present your recommendations in a structured, easy-to-read format.
- For each selected resource, briefly explain why it is specifically relevant.

## Selected Most Relevant Resources (clearly listed and explained):
"""
    messages = [
        {"role": "system", "content": "You are an expert assistant selecting highly relevant resources tailored to the user's situation."},
        {"role": "user", "content": prompt}
    ]

    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=messages
    )
    return response.choices[0].message.content

In [60]:
def format_selected_resources(raw_text):
    formatted_text = raw_text.replace("\\n", "\n")
    formatted_text = re.sub(r'\*\*', '', formatted_text)
    formatted_text = re.sub(r'###+', '', formatted_text)
    formatted_text = re.sub(r'\s*(\d+)\.\s*(.+?):\s*\n', r'\n\1. **\2**\n', formatted_text)
    formatted_text = re.sub(r'–\s*URL\s*:', '- **URL:**', formatted_text, flags=re.I)
    formatted_text = re.sub(r'–\s*Phone\s*:', '- **Phone:**', formatted_text, flags=re.I)
    formatted_text = re.sub(r'–\s*Relevance\s*:', '- **Relevance:**', formatted_text, flags=re.I)
    formatted_text = re.sub(r'\n+', '\n', formatted_text).strip()
    return formatted_text

In [61]:
selected_resources = select_relevant_resources(
        goal,
        situation,
        resource_reasoning,
        retrieved_resources)
resource_output = format_selected_resources(selected_resources)
print(resource_output)

## Selected Most Relevant Resources
1. Affordable Housing Alliance  
   - URL: https://housingall.org/  
   - Phone: 7323892958  
   - Relevance: This organization specializes in affordable housing and housing stability programs in New Jersey, which is highly relevant given the user’s temporary housing situation. They provide support for rental assistance and other housing subsidies, fitting the user's needs for stable living conditions. Their services are tailored for residents facing financial challenges, complementing the user’s part-time employment and physical disability context.
2. Clear Conscience Counseling, LLC  
   - URL: https://www.clearconsciencenj.com/  
   - Phone: 9734347830  
   - Relevance: This mental health counseling service offers flexible, personalized therapy for underserved populations, which fits the user’s need for support balancing work, wellness, and social engagement. Their evidence-based outpatient services and community support can help address emotional